In [ ]:
from pyspark import SparkContext

In [1]:
!pip install findspark

In [1]:
import findspark

In [2]:
findspark.init("/home/ubuntu/spark-2.4.4-bin-hadoop2.7")

In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext()

/home/ubuntu/anaconda3/lib/python3.5/site-packages/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [6]:
!python --version

Python 3.5.2 :: Anaconda 4.1.1 (64-bit)


In [7]:
%%writefile example.txt
First line
Second line
Third line
Fourth line

Writing example.txt


In [8]:
textFile = sc.textFile('example.txt')

In [9]:
textFile.count()

4

In [10]:
#RDD => it distrubutes the objects across multiple servers

In [11]:
textFile.first()

'First line'

In [16]:
secfind = textFile.filter(lambda line: "Second" in line)

In [32]:
secfind.collect()[0].split()

['Second', 'line']

In [36]:
sec = secfind.flatMap(lambda x: x.split('n'))

In [37]:
sec.collect()

['Seco', 'd li', 'e']

In [39]:
secfind.distinct()

In [40]:
sec.collect()

['Second line']

In [41]:
#RDD Transformation and actions

In [42]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

Writing example2.txt


In [45]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[22] at textFile at NativeMethodAccessorImpl.java:0

In [46]:
text_rdd = sc.textFile('example2.txt')

In [47]:
text_map = text_rdd.map(lambda line: line.split())

In [48]:
text_map.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [49]:
#map vs flatmap

In [50]:
text_flatMap = text_rdd.flatMap(lambda line: line.split()).collect()

In [53]:
text_flatMap

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [1]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [21]:
services = sc.textFile('services.txt')

In [22]:
services.collect()

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [55]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [56]:
services.map(lambda line: line.split()).collect()

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [57]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

In [58]:
clean = clean.map(lambda line: line.split())

In [59]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [60]:
#Using reduced by key statment

In [67]:
pairs = clean.map(lambda lst: (lst[3], lst[-1]))

In [62]:
rekey = pairs.reduceByKey(lambda amnt1,amnt2: float(amnt1) + float(amnt2))

In [68]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [64]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [82]:
#Grabe (State, Amount)
step1 = clean.map(lambda lst: (lst[3], lst[-1]))

#Reduced by key
step2 = step1.reduceByKey(lambda amt1,amt2 : float(amt1) + float(amt2))

#Get Rid of states, amount titles
step3 = step2.filter(lambda x: not x[0] == 'State')

#Sort results by amount
step4 = step3.sortBy(lambda stAmount : stAmount[1], ascending=False)

#Actino
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [83]:
X = ['ID', 'State', 'Amount']

In [84]:
def func1(lst):
    return lst[-1]

In [85]:
def func2(id_st_amt):
    #unpack the values
    (Id,st,amt) = id_st_amt
    return amt

In [86]:
func1(X)

'Amount'

In [87]:
func2(X)

'Amount'